In [1]:
import numpy as np
import pandas as pd
import os
import re
from tqdm import tqdm
import requests
from io import StringIO
from os import listdir
from os.path import isfile, join
import datetime as dt
from datetime import datetime, timedelta

import gridstatus

In [2]:
def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f



In [3]:
FILE_PATH = "/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data"

In [5]:
ISOS = ["MISO"]

In [6]:
ISO_files = {}
ISO_data = {}
ISO_final = {}

#load in
for ISO in ISOS:
    ISO_files[ISO] = listdir_nohidden(f"{FILE_PATH}/{ISO}/old")
    ISO_data[ISO] = {}
    for file in tqdm(ISO_files[ISO]):
        compiled = re.compile(re.escape(f"{ISO}_"), re.IGNORECASE)

        ISO_data[ISO][compiled.sub("", file).replace(".csv","")] = pd.read_csv(f"{FILE_PATH}/{ISO}/old/{file}")

16it [00:00, 20.11it/s]


In [7]:
#add merge datetime
titles = {}
for key in ISO_data:
    print(key)
    titles[key]= []
    for k in ISO_data[key]:
        try:
            ISO_data[key][k]["time_tomerge"] = pd.to_datetime(ISO_data[key][k].iloc[:,0], utc=True)
        except:
            ISO_data[key][k] = ISO_data[key][k][ISO_data[key][k].index % 24 != 0]
            ISO_data[key][k]["time_tomerge"] = pd.to_datetime(ISO_data[key][k].iloc[:,0], utc=True)
            print(type(ISO_data[key][k]["time_tomerge"]))

MISO
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [9]:
ISO_final = {}
def clean_final(ISO: str, df: str):
    ISO_data[ISO][df].columns = ISO_data[ISO][df].columns.str.lower()
    no_date = [col for col in ISO_data[ISO][df].columns if "date" not in col]
    ISO_data[ISO][df] = ISO_data[ISO][df][no_date]
    try:
        ISO_data[ISO][df] = ISO_data[ISO][df].drop("unnamed: 0")
        ISO_data[ISO][df] = ISO_data[ISO][df].drop("hour")
    except:
        pass
    ISO_data[ISO][df] = ISO_data[ISO][df].sort_values(by="time_tomerge")
    ISO_data[ISO][df] = ISO_data[ISO][df].set_index("time_tomerge")
    ISO_final[ISO] = ISO_data[ISO][df]
    for column in ISO_final[ISO].columns:
        if str(ISO).lower() in column:
            ISO_final[ISO] = ISO_final[ISO].rename(columns={column: column.replace(f"{str(ISO).lower()}_","")})
    print(ISO_final[ISO])

In [11]:
for iso in ISOS:
    clean_final(iso, "final")

                           congestion_da  congestion_rt  energy_da  energy_rt  \
time_tomerge                                                                    
2014-01-01 06:00:00+00:00      -0.824286      -1.292857      30.10      27.68   
2014-01-01 07:00:00+00:00      -0.365714      -1.008571      28.72      26.11   
2014-01-01 08:00:00+00:00      -0.701429      -0.837143      28.50      24.97   
2014-01-01 09:00:00+00:00      -0.754286       0.330000      28.35      24.15   
2014-01-01 10:00:00+00:00      -0.361429       0.477143      28.48      23.67   
...                                  ...            ...        ...        ...   
2019-01-01 01:00:00+00:00       1.427500       3.543750      25.76      19.70   
2019-01-01 02:00:00+00:00       1.225000       3.352500      24.07      19.11   
2019-01-01 03:00:00+00:00       1.396250       2.673750      22.51      18.89   
2019-01-01 04:00:00+00:00       1.218750      -3.087500      21.88      21.81   
2019-01-01 05:00:00+00:00   

In [58]:
FILE_PATH = "/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data/MISO/new"
generation_rt = pd.read_csv(f"{FILE_PATH}/combined/generation_rt.csv", thousands = ",")
generation_rt.columns

Index(['Unnamed: 0', 'Market Hour Ending', 'Coal', 'Gas', 'Nuclear', 'Hydro',
       'Wind', 'Other', 'Total MW', 'Unnamed: 8', 'Coal.1', 'Gas.1',
       'Nuclear.1', 'Hydro.1', 'Wind.1', 'Other.1', 'Total MW.1',
       'Unnamed: 16', 'Coal.2', 'Gas.2', 'Nuclear.2', 'Hydro.2', 'Other.2',
       'Total MW.2', 'Unnamed: 23', 'Sum:', 'HE', 'Coal.3', 'Gas.3',
       'Nuclear.3', 'Hydro.3', 'Wind.2', 'Other.3', 'MISO', 'Solar',
       'Unnamed: 9', 'Solar.1', 'Unnamed: 18', 'Solar.2', 'Unnamed: 26',
       'Solar.3', 'Storage', 'Unnamed: 10', 'Storage.1', 'Unnamed: 20',
       'Unnamed: 28', 'Storage.2', 'Unnamed: 1', 'Unnamed: 4', 'Unnamed: 19',
       'Unnamed: 27'],
      dtype='object')

In [72]:
MISO_gen = generation_rt[["Market Hour Ending", "MISO"]]
MISO_gen = MISO_gen[MISO_gen['Market Hour Ending'] == MISO_gen["Market Hour Ending"][24]]
MISO_gen = MISO_gen.dropna()
print(np.sum(MISO_gen["MISO"][:350]))

582940711.6000001


In [51]:
MISO = pd.read_csv("/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/ISOs/MISO.csv", thousands = ",")

In [136]:
len(MISO)

78888

In [74]:
year = {}
MISO["time_tomerge"] = pd.to_datetime(MISO["time_tomerge"])
for year in np.unique(MISO["time_tomerge"].apply(lambda x: x.year)):
    print(np.sum(np.sum(MISO[[col for col in MISO.columns if "load_rt" in col]][MISO["time_tomerge"].apply(lambda x: x.year) == year])))

677378680.16
667757752.63
665012374.83
655069361.42
681903495.0
667860553.7
639828861.4
660172343.0999999
670156753.6999999


In [53]:
miso2014 = MISO[[col for col in MISO.columns if "gen_rt" in col]]


print((miso2014.astype(float)))

       gen_rt_coal  gen_rt_gas  gen_rt_hydro  gen_rt_nuclear  gen_rt_other  \
0          47208.1      7950.4         493.8         11608.3        1438.3   
1          45614.0      7553.1         493.2         11607.9        1408.1   
2          44471.2      7440.9         493.4         11608.9        1395.7   
3          43614.5      7356.7         492.6         11615.3        1430.6   
4          43167.2      7404.9         492.8         11617.2        1414.6   
...            ...         ...           ...             ...           ...   
78883      16782.1      7986.8         573.2          4671.9         366.7   
78884      16191.7      7603.4         523.1          4669.3         365.1   
78885      15677.8      7466.0         517.6          4669.2         366.1   
78886      15230.5      6967.3         508.0          4671.2         366.8   
78887      14508.7      6461.3         496.3          4673.0         364.3   

       gen_rt_wind  
0           3866.6  
1           3829.9  


In [54]:
MISO.columns

Index(['Unnamed: 0', 'time_tomerge', 'congestion_da', 'congestion_rt',
       'energy_da', 'energy_rt', 'exchange_rt', 'gas_price', 'gen_da_coal',
       'gen_da_gas', 'gen_da_hydro', 'gen_da_nuclear', 'gen_da_other',
       'gen_da_wind', 'gen_rt_coal', 'gen_rt_gas', 'gen_rt_hydro',
       'gen_rt_nuclear', 'gen_rt_other', 'gen_rt_wind', 'load_da', 'load_rt',
       'losses_da', 'losses_rt'],
      dtype='object')

In [84]:
test = pd.read_csv("/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data/MISO/new/generation_rt/generation_rt_2021_01_02.csv")

In [85]:
print(test)

   Real-Time State Estimator Generation Fuel Mix Report Unnamed: 1 Unnamed: 2  \
0                           Publish Date:  2021-01-02          NaN        NaN   
1                            Market Date:  2021-01-01          NaN        NaN   
2                                                 NaN      Central        NaN   
3                                  Market Hour Ending         Coal        Gas   
4                                                   1     17,300.8    6,193.6   
5                                                   2     17,266.3    6,268.8   
6                                                   3     17,728.7    6,210.1   
7                                                   4     17,363.2    6,036.6   
8                                                   5     17,429.0    5,756.5   
9                                                   6     17,660.1    6,041.0   
10                                                  7     18,729.6    6,034.9   
11                          

In [96]:
test1 = test[test.eq("Total").any(1)].reset_index()

In [97]:
test2 = test1.iloc[0,:]

In [118]:
test2.index[test2 == "Total"][0]

'Unnamed: 24'

In [126]:
ISO_done = ["MISO"]
FILE_PATH = "/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data/MISO/new"
for file in (listdir_nohidden(FILE_PATH)):
    if file in ["generation_rt", "generation_da"]:
        dfs = []
        for f in tqdm(listdir_nohidden(f"{FILE_PATH}/{file}")):
            temp = pd.read_csv(f"{FILE_PATH}/{file}/{f}")
            temp1 = temp[temp.eq("Total").any(1)].reset_index()
            temp2 = temp1.iloc[0,:]
            col = int(test2.index[test2 == "Total"][0].replace("Unnamed: ", ""))
            temp = temp.iloc[3:,(col+1):]
            temp.columns = temp.iloc[0,:]
            temp = temp.iloc[1:,:]
            temp = temp[temp["HE"]!="Sum:"]
            temp = temp.dropna()
            temp["date"] = [f[-14:-4].replace("_","-") for _ in range(len(temp))]

        temp = pd.concat(dfs)
        os.makedirs(f"{FILE_PATH}/combined", exist_ok=True)
        temp.to_csv(f"{FILE_PATH}/combined/{file}.csv")

438it [00:03, 111.15it/s]


KeyError: 'HE'